In [1]:
import tensorflow as tf
import numpy as np
import os
import time

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
from sklearn.metrics import classification_report

In [2]:
# PATH DATASET
DATASET_DIR = "D:/KULIAH/SEMESTER 7/Skripsi/Dataset/Dataset_TrashNet_Final"
TRAIN_DIR = os.path.join(DATASET_DIR, "train")
TEST_DIR  = os.path.join(DATASET_DIR, "test")

# PATH MODEL
SAVED_MODEL_PATH = "D:/KULIAH/SEMESTER 7/Skripsi/Dataset/mobilenetv3Large_trashnet_base"
TFLITE_INT8_PATH = "D:/KULIAH/SEMESTER 7/Skripsi/Dataset/Quantisasi/mobilenetv3large_ptq_int8.tflite"

# PARAMETER
IMG_SIZE = (224, 224)
BATCH_SIZE = 1
CALIBRATION_SAMPLES = 100

In [3]:
model_fp32 = tf.keras.models.load_model(
    SAVED_MODEL_PATH,
    compile=False
)

print("Model FP32 berhasil dimuat.")




Model FP32 berhasil dimuat.


In [4]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

class_names = list(test_generator.class_indices.keys())

Found 2001 images belonging to 6 classes.
Found 383 images belonging to 6 classes.


In [5]:
# REPRESENTATIVE DATASET (KALIBRASI INT8 → DATA TRAIN)
def representative_dataset():
    for i in range(CALIBRATION_SAMPLES):
        image, _ = next(train_generator)
        yield [image.astype(np.float32)]

In [6]:
converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL_PATH)

# Optimasi default
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Kalibrasi aktivasi
converter.representative_dataset = representative_dataset

# Full integer quantization
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS_INT8
]

# Input dan output INT8
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

In [7]:
print("Melakukan Post-Training Quantization INT8...")
tflite_int8_model = converter.convert()

with open(TFLITE_INT8_PATH, "wb") as f:
    f.write(tflite_int8_model)

model_size_mb = os.path.getsize(TFLITE_INT8_PATH) / (1024 * 1024)

print(f"Model INT8 berhasil disimpan.")
print(f"Ukuran model INT8: {model_size_mb:.2f} MB")

Melakukan Post-Training Quantization INT8...
Model INT8 berhasil disimpan.
Ukuran model INT8: 3.40 MB


In [8]:
# LOAD INTERPRETER TFLITE
interpreter = tf.lite.Interpreter(model_path=TFLITE_INT8_PATH)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [9]:
# EVALUASI MODEL (DATA TEST SAJA)
y_true = []
y_pred = []

start_time = time.time()

for i in range(len(test_generator)):
    image, label = test_generator[i]

    # Quantize input
    scale, zero_point = input_details[0]["quantization"]
    image_int8 = (image / scale + zero_point).astype(np.int8)

    interpreter.set_tensor(
        input_details[0]["index"], image_int8
    )
    interpreter.invoke()

    output = interpreter.get_tensor(
        output_details[0]["index"]
    )

    y_true.append(np.argmax(label))
    y_pred.append(np.argmax(output))

end_time = time.time()

In [11]:
# LAPORAN WAKTU INFERENSI
total_time = end_time - start_time
avg_inference_time = total_time / len(test_generator)

print("=== Time Report PTQ INT8 ===")
print(f"Total inference time  : {total_time:.4f} detik")
print(f"Rata-rata per gambar  : {avg_inference_time:.6f} detik")

=== Time Report PTQ INT8 ===
Total inference time  : 41.3195 detik
Rata-rata per gambar  : 0.107884 detik


In [12]:
# CLASSIFICATION REPORT
print("Classification Report PTQ INT8")
print(
    classification_report(
        y_true,
        y_pred,
        target_names=class_names,
        digits=4
    )
)

Classification Report PTQ INT8
              precision    recall  f1-score   support

   cardboard     0.6000    0.5902    0.5950        61
       glass     0.7222    0.1711    0.2766        76
       metal     0.7429    0.4194    0.5361        62
       paper     0.3496    0.9556    0.5119        90
     plastic     0.7500    0.0833    0.1500        72
       trash     0.0625    0.0455    0.0526        22

    accuracy                         0.4386       383
   macro avg     0.5379    0.3775    0.3537       383
weighted avg     0.5859    0.4386    0.3880       383

